In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import tensorflow as tf
import cv2
import mediapipe as mp
from keras.models import load_model
import numpy as np
import time
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

In [ ]:
from keras.models import load_model
import numpy as np
import time
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

In [ ]:
model = load_model('smnist.h5')

base_options = python.BaseOptions(model_asset_path='hand_landmarker.task')
options = vision.HandLandmarkerOptions(base_options=base_options, num_hands=2)
detector = vision.HandLandmarker.create_from_options(options)

cap = cv2.VideoCapture(0)

# Check if camera opened successfully
if not cap.isOpened():
    print("Error: Could not open camera. Check permissions or index.")
    exit()

# Try to read a frame
ret, frame = cap.read()

if not ret or frame is None:
    print("Error: Successfully opened camera but failed to read a frame.")
    cap.release()
    exit()

# Now it is safe to get the shape
h, w, c = frame.shape
print(f"Camera Initialized: {w}x{h}")

analysisframe = ''
letterpred = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y']


W0000 00:00:1767409777.865344   12236 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1767409777.873893   12236 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


Error: Successfully opened camera but failed to read a frame.


[ WARN:0@22.370] global cap_v4l.cpp:804 requestBuffers VIDEOIO(V4L2:/dev/video0): failed VIDIOC_REQBUFS: errno=19 (No such device)


AttributeError: 'NoneType' object has no attribute 'shape'

: 

In [ ]:
while cap.isOpened():
    ret, frame = cap.read()
    if not ret: break
    
    h, w, c = frame.shape
    # Flip for mirror effect and convert to RGB for MediaPipe
    frame = cv2.flip(frame, 1)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=rgb_frame)

    # 3. Perform Landmark Detection
    detection_result = detector.detect(mp_image)

    # 4. Process Results
    if detection_result.hand_landmarks:
        for hand_landmarks in detection_result.hand_landmarks:
            # Get bounding box coordinates from landmarks
            x_coords = [lm.x for lm in hand_landmarks]
            y_coords = [lm.y for lm in hand_landmarks]
            
            x_min, x_max = int(min(x_coords) * w), int(max(x_coords) * w)
            y_min, y_max = int(min(y_coords) * h), int(max(y_coords) * h)

            # Add padding for the crop (similar to your old code)
            x_min, x_max = max(0, x_min - 20), min(w, x_max + 20)
            y_min, y_max = max(0, y_min - 20), min(h, y_max + 20)

            # 5. Prepare Image for Keras Model
            try:
                # Crop and convert to Grayscale (smnist usually requires 28x28 grayscale)
                hand_roi = frame[y_min:y_max, x_min:x_max]
                gray_roi = cv2.cvtColor(hand_roi, cv2.COLOR_BGR2GRAY)
                resized_roi = cv2.resize(gray_roi, (28, 28))
                
                # Normalize and reshape for model: (1, 28, 28, 1)
                img_array = resized_roi.reshape(1, 28, 28, 1) / 255.0
                
                # Make Prediction
                prediction = model.predict(img_array, verbose=0)
                result_idx = np.argmax(prediction)
                confidence = np.max(prediction)
                
                label = f"{letterpred[result_idx]} ({confidence:.2f})"
                
                # Draw Visuals
                cv2.rectangle(frame, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)
                cv2.putText(frame, label, (x_min, y_min - 10), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            except Exception as e:
                pass # Handle cases where ROI is too small

    cv2.imshow("e-salinify", frame)

    if cv2.waitKey(1) & 0xFF == 27: # ESC to exit
        break

cap.release()
cv2.destroyAllWindows()

W0000 00:00:1767409757.834841   11764 landmark_projection_calculator.cc:78] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.
qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in "/home/haise/Documents/e-salinify/.venv/lib/python3.12/site-packages/cv2/qt/plugins"
[ WARN:0@2.191] global cap_v4l.cpp:804 requestBuffers VIDEOIO(V4L2:/dev/video0): failed VIDIOC_REQBUFS: errno=19 (No such device)
